In [ ]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
import re
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()


## first calculate introductions

In [ ]:
from datetime import date
current_date = str(date.today())

In [ ]:
def read_in_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
                if "sample" in line.lower():
                    all_cols = line.split("\t")
                    migration_column_indices = []   # list to store column indices
                    mig_rates_key = {}   # dictionary to store the column index to map to column name
                    counter = 0
                    for i in range(len(all_cols)):
                        
                        col = all_cols[i]
                        if col == "immigrationRate.1":
                            counter = counter + 1
                        if ("immigrationRate" in col) and (counter <2):
                            migration_column_indices.append(i)


                    # make an empty dictionary to store migration rates and generate dictionary to convert index to name
                    for m in migration_column_indices:
                        name = line.split("\t")[m]
                        mig_rates_key[m] = name
                        mig_rates_dict[name] = []
                    
                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in migration_column_indices:
                        name = mig_rates_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                
                
    return(mig_rates_dict)

In [ ]:
log_file_path = "../../mascot_glm/results/glm_random_3000_downsampled.log"


In [ ]:
migration_rates = read_in_migration_rates_mascot(log_file_path)
mig_df = pd.DataFrame.from_dict(migration_rates)

In [ ]:
burnin_percent = 0.1

mig_df = pd.DataFrame.from_dict(migration_rates)
print(len(mig_df))

rows_to_remove = int(len(mig_df)* burnin_percent)
mig_df = mig_df.iloc[rows_to_remove:]

print(len(mig_df))
mig_df = mig_df.reset_index()
mig_df.head()

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_mr_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
            #deme = i.split("_")[1]
            interval = i.split(".")[1]
            next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            mean_log = local_series.mean()
            mean_linear = np.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_mr_log":mean_log,"mean_mr_linear":mean_linear, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50,"lower_hpd_linear_50":lower_hpd_linear_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [ ]:
mr_summary = generate_summary_mr_df(mig_df)
mr_summary['days'] = mr_summary.interval.astype(int) *14
mr_summary['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - mr_summary.days.map(timedelta)
mr_summary.date = mr_summary.date.astype(str)

In [ ]:
mr_summary

In [ ]:
line = alt.Chart(mr_summary).mark_area(interpolate='monotone').encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="introductions", grid=False)),
    alt.Y2('upper_hpd_linear_50' )
).properties(
    width=850,
    height=300
)

band = alt.Chart(mr_summary).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_linear'),
    alt.Y2('upper_hpd_linear')
).properties(
    width=850,
    height=300
)

band + line

## now working on Ne diff

In [ ]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
            
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        interval = name.split(".")[2]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            deme = i.split(".")[1]
            #print(deme)
            interval = i.split(".")[2]
            #print(interval)
            #print(i)
            next_interval = int(interval)+7
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            mean_linear = 10**mean_log
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = 10**lower_hpd_log_95
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = 10**upper_hpd_log_95
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = 10**lower_hpd_log_50
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = 10**upper_hpd_log_50
            
            try:
                next_local_series = input_df["Ne"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
                diff_series = np.subtract(local_series, next_local_series)
                #print(local_series)
                #print(next_local_series)
                #print(diff_series)
                diff_mean_log = diff_series.mean()
                diff_hpd_95 = az.hdi(diff_series, 0.95)
                diff_lower_hpd_log_95 = diff_hpd_95[0]
                diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
                diff_upper_hpd_log_95 = diff_hpd_95[1]
                diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
                diff_hpd_50 = az.hdi(diff_series, 0.50)
                diff_lower_hpd_log_50 = diff_hpd_50[0]
                diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
                diff_upper_hpd_log_50 = diff_hpd_50[1]
                diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
            except KeyError:
                pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "diff_mean_Ne_log":diff_mean_log, 
                                                   "diff_upper_hpd_log_95":diff_upper_hpd_log_95,"diff_lower_hpd_log_95":diff_lower_hpd_log_95, 
                                                   "diff_upper_hpd_log_50":diff_upper_hpd_log_50,"diff_lower_hpd_log_50":diff_lower_hpd_log_50,
                                                   "diff_upper_hpd_linear":diff_upper_hpd_linear_95,"diff_lower_hpd_linear":diff_lower_hpd_linear_95,
                                                   "diff_upper_hpd_linear_50":diff_upper_hpd_linear_50,"diff_lower_hpd_linear_50":diff_lower_hpd_linear_50})
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [ ]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

In [14]:
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
burnin_percent = 0.1

rows_to_remove = int(len(Ne_df)* burnin_percent)
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()
Ne_df

4936
4443


,index,sample,Ne.0.0,Ne.0.1,Ne.0.2,Ne.0.3,Ne.0.4,Ne.0.5,Ne.0.6,Ne.0.7,...,Ne.1.796,Ne.1.797,Ne.1.798,Ne.1.799,Ne.1.800,Ne.1.801,Ne.1.802,Ne.1.803,Ne.1.804,Ne.1.805
0,493,2465000,3.67520909348397,3.7147739673266646,3.8794609236125237,3.6681160460205735,3.836057590141957,4.133848176725926,4.559930548721714,4.861700380883387,...,0.06701049534808212,0.0634277103871301,0.060037086350178806,0.05682771324109092,0.05378936118432957,0.053195875521162,0.05293953441979973,0.05268442857887084,0.0524305056945393,0.05217780664109901
1,494,2470000,5.909090007635588,5.823905569064001,5.854869337728972,5.541439962460016,5.575620089339903,5.713073101327605,5.937382696160703,6.044416540085496,...,0.6089601319751146,0.600027934608381,0.5912283528706949,0.5825578195227439,0.5740128899809149,0.5628830164775187,0.551592436728966,0.5405283288890117,0.5296841860203009,0.5190575996204561
2,495,2475000,8.332325123245475,8.098001521606818,8.027818609657848,7.492379896168627,7.433755261838808,7.511087009513923,7.697414406054761,7.727179933222485,...,0.7263217069294633,0.7243191397973697,0.7223224589341006,0.7203312821839366,0.7183452314626683,0.6971631076885418,0.6739892062686543,0.6515856119707282,0.6299228222746424,0.6089802394842818
3,496,2480000,8.643176151848335,8.555421270668884,8.638114187072265,8.211056192254224,8.297435435544763,8.538759946432656,8.912401113726089,9.112315031168938,...,0.9731656944292337,0.9759034941239986,0.9786484929649524,0.9814012128753384,0.9841621814919184,0.9624789803503913,0.9379111863005397,0.9139704984179936,0.8906366947624808,0.8678986066075991
4,497,2485000,7.665772626396549,7.641371977662197,7.769566954168309,7.437463163406162,7.56862622407214,7.843598478114343,8.244478773880608,8.488777225928363,...,1.1156974809378504,1.1186053640232665,1.1215202918572036,1.1244428155820159,1.1273734919524596,1.1017156853709074,1.0727138311942133,1.044475429473411,1.016975417681808,0.9901994541800827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,4931,24655000,3.049063504048463,3.0255346272270813,3.0883231047728668,2.8924356639209847,2.9560234207368983,3.1005071471312187,3.318423349804083,3.448560885701086,...,0.24261973504231127,0.242194008287806,0.24176910625502912,0.24134494966487782,0.24092145978628596,0.23714751305231443,0.23296606375519183,0.22885834290666024,0.22482231853393278,0.22085747134675196
4439,4932,24660000,3.6568274520337867,3.6525359541390876,3.7529259712483594,3.5380655134364214,3.63969009035843,3.842763397112297,4.13997381732019,4.330704959487055,...,0.2040107153076234,0.2022874007655024,0.20057895471447587,0.1988849375794281,0.1972049213870306,0.194439864252165,0.1915597988534301,0.18872239331116542,0.1859265079520786,0.18317204308795182
4440,4933,24665000,4.937137954305975,4.885680673927266,4.979153919516588,4.752022062855423,4.9089127687451555,5.1424022169914485,5.482667966940708,5.720266082604066,...,0.2784556438124749,0.27555010072676295,0.27267539886833175,0.2698306876023661,0.2670151415500698,0.26470348700940677,0.2624795062005105,0.2602742107920007,0.25808704526593945,0.2559182591752577
4441,4934,24670000,4.123489862672763,4.1432291230236284,4.268732868410629,4.075104643047718,4.203023016899323,4.435882618132676,4.766214921335143,4.989212485996879,...,0.22638153547933348,0.22384935425932107,0.2213459522375981,0.218870546819676,0.21642237944994305,0.21382557503745872,0.21123557298426815,0.2086769428127359,0.20614884474014333,0.20365137429597166


## calculating transmission rate

In [15]:
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne" in i:
            deme = i.split(".")[1]
            interval = i.split(".")[2]
            next_interval = int(interval)+7
            local_series = input_df[i].astype('float').to_numpy()
           
            try:
                new_df["Ne"+"."+ str(deme) +".diff." + str(interval)] = (365/7)*(np.log(input_df[i].astype("float")) - np.log(input_df["Ne"+"."+ str(deme) +"." + str(next_interval)].astype('float')))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [16]:
ne_diff_summary = generate_summary_diff_df(Ne_df)

In [17]:
ne_diff_summary

,Ne.0.diff.0,Ne.0.diff.1,Ne.0.diff.2,Ne.0.diff.3,Ne.0.diff.4,Ne.0.diff.5,Ne.0.diff.6,Ne.0.diff.7,Ne.0.diff.8,Ne.0.diff.9,...,Ne.1.diff.789,Ne.1.diff.790,Ne.1.diff.791,Ne.1.diff.792,Ne.1.diff.793,Ne.1.diff.794,Ne.1.diff.795,Ne.1.diff.796,Ne.1.diff.797,Ne.1.diff.798
0,-14.588436,-18.443022,-20.267926,-21.428465,-19.652624,-19.744234,-19.363744,-18.522026,-16.522441,-14.769943,...,20.054075,20.054599,20.054599,20.054075,20.054075,17.767948,15.155179,12.541886,9.928639,7.315916
1,-1.180677,-3.519956,-4.627657,-5.331968,-4.254041,-4.309638,-4.078925,-3.568102,-2.354389,-1.290830,...,5.392891,5.393032,5.393032,5.392891,5.392891,5.643497,5.929683,6.215729,6.501967,6.788347
2,3.931497,1.592218,0.484382,-0.219928,0.858132,0.802536,1.033114,1.543938,2.757784,3.821344,...,1.007639,1.007665,1.007665,1.007639,1.007639,2.424383,4.043151,5.661893,7.280957,8.900047
3,-2.756098,-5.095377,-6.203038,-6.907348,-5.829463,-5.885059,-5.654306,-5.143482,-3.929811,-2.866251,...,-1.025303,-1.025330,-1.025330,-1.025303,-1.025303,0.282821,1.777537,3.272281,4.767271,6.262234
4,-5.317511,-7.656790,-8.764383,-9.468693,-8.390875,-8.446472,-8.215651,-7.704828,-6.491223,-5.427664,...,-0.949976,-0.950001,-0.950001,-0.949976,-0.949976,0.386152,1.912865,3.439602,4.966594,6.493561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,-6.419960,-9.757740,-11.338132,-12.343071,-10.805165,-10.884492,-10.555178,-9.826314,-8.094662,-6.577131,...,0.640964,0.640981,0.640981,0.640964,0.640964,1.372669,2.208709,3.044732,3.880925,4.717135
4439,-8.819150,-12.156931,-13.737259,-14.742198,-13.204355,-13.283682,-12.954306,-12.225442,-10.493852,-8.976321,...,3.095992,3.096073,3.096073,3.095992,3.095992,3.390023,3.725896,4.061689,4.397624,4.733639
4440,-7.676968,-11.379035,-13.278579,-13.107830,-11.151478,-11.331492,-10.591071,-10.436195,-8.608052,-7.309195,...,3.828199,3.828299,3.828299,3.828199,3.828199,3.734743,3.627844,3.520845,3.413926,3.307107
4441,-9.937293,-12.894736,-14.294931,-15.185358,-13.822806,-13.893095,-13.601195,-12.955384,-11.421164,-10.076555,...,4.105268,4.105375,4.105375,4.105268,4.105268,4.148282,4.197307,4.246225,4.295260,4.344402


In [18]:
uninfectious_rate = 365/7

#taken from https://www.medrxiv.org/content/10.1101/2020.09.12.20193284v1.full.pdf 


In [19]:
ne_diff_summary += uninfectious_rate

In [20]:
ne_diff_summary

,Ne.0.diff.0,Ne.0.diff.1,Ne.0.diff.2,Ne.0.diff.3,Ne.0.diff.4,Ne.0.diff.5,Ne.0.diff.6,Ne.0.diff.7,Ne.0.diff.8,Ne.0.diff.9,...,Ne.1.diff.789,Ne.1.diff.790,Ne.1.diff.791,Ne.1.diff.792,Ne.1.diff.793,Ne.1.diff.794,Ne.1.diff.795,Ne.1.diff.796,Ne.1.diff.797,Ne.1.diff.798
0,37.554421,33.699835,31.874931,30.714392,32.490233,32.398623,32.779114,33.620831,35.620417,37.372914,...,72.196932,72.197456,72.197456,72.196932,72.196932,69.910805,67.298036,64.684743,62.071496,59.458773
1,50.962180,48.622901,47.515200,46.810890,47.888816,47.833219,48.063932,48.574755,49.788468,50.852027,...,57.535748,57.535889,57.535889,57.535748,57.535748,57.786354,58.072541,58.358586,58.644824,58.931204
2,56.074354,53.735075,52.627240,51.922929,53.000989,52.945393,53.175972,53.686795,54.900641,55.964201,...,53.150496,53.150522,53.150522,53.150496,53.150496,54.567240,56.186008,57.804750,59.423814,61.042904
3,49.386759,47.047480,45.939820,45.235510,46.313394,46.257798,46.488552,46.999375,48.213047,49.276606,...,51.117554,51.117528,51.117528,51.117554,51.117554,52.425678,53.920394,55.415138,56.910128,58.405091
4,46.825346,44.486067,43.378474,42.674164,43.751982,43.696385,43.927206,44.438029,45.651634,46.715193,...,51.192881,51.192856,51.192856,51.192881,51.192881,52.529009,54.055722,55.582459,57.109451,58.636419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,45.722897,42.385117,40.804725,39.799786,41.337693,41.258365,41.587679,42.316543,44.048195,45.565726,...,52.783821,52.783838,52.783838,52.783821,52.783821,53.515526,54.351566,55.187589,56.023782,56.859992
4439,43.323707,39.985926,38.405598,37.400659,38.938502,38.859175,39.188552,39.917415,41.649005,43.166536,...,55.238849,55.238930,55.238930,55.238849,55.238849,55.532880,55.868754,56.204546,56.540481,56.876496
4440,44.465889,40.763822,38.864278,39.035027,40.991379,40.811366,41.551786,41.706663,43.534805,44.833662,...,55.971056,55.971156,55.971156,55.971056,55.971056,55.877600,55.770701,55.663702,55.556783,55.449965
4441,42.205564,39.248121,37.847926,36.957499,38.320051,38.249763,38.541663,39.187473,40.721693,42.066303,...,56.248125,56.248233,56.248233,56.248125,56.248125,56.291139,56.340164,56.389082,56.438117,56.487259


In [21]:
north_ne_diff =  ne_diff_summary.filter(regex='Ne.0.')
south_ne_diff =  ne_diff_summary.filter(regex='Ne.1.')



In [22]:
south_ne_diff

,Ne.1.diff.0,Ne.1.diff.1,Ne.1.diff.2,Ne.1.diff.3,Ne.1.diff.4,Ne.1.diff.5,Ne.1.diff.6,Ne.1.diff.7,Ne.1.diff.8,Ne.1.diff.9,...,Ne.1.diff.789,Ne.1.diff.790,Ne.1.diff.791,Ne.1.diff.792,Ne.1.diff.793,Ne.1.diff.794,Ne.1.diff.795,Ne.1.diff.796,Ne.1.diff.797,Ne.1.diff.798
0,42.347122,37.188915,35.291947,30.895427,29.709574,31.314708,31.655452,25.197289,31.639444,30.397565,...,72.196932,72.197456,72.197456,72.196932,72.196932,69.910805,67.298036,64.684743,62.071496,59.458773
1,53.870784,50.740361,49.588925,46.920756,46.201284,47.175411,47.382002,43.462659,47.372488,46.618813,...,57.535748,57.535889,57.535889,57.535748,57.535748,57.786354,58.072541,58.358586,58.644824,58.931204
2,58.982957,55.852535,54.700965,52.032796,51.313458,52.287585,52.494042,48.574699,52.484661,51.730987,...,53.150496,53.150522,53.150522,53.150496,53.150496,54.567240,56.186008,57.804750,59.423814,61.042904
3,52.295363,49.164940,48.013545,45.345376,44.625863,45.599990,45.806622,41.887279,45.797066,45.043392,...,51.117554,51.117528,51.117528,51.117554,51.117554,52.425678,53.920394,55.415138,56.910128,58.405091
4,49.733950,46.603527,45.452199,42.784030,42.064450,43.038577,43.245276,39.325933,43.235654,42.481979,...,51.192881,51.192856,51.192856,51.192881,51.192881,52.529009,54.055722,55.582459,57.109451,58.636419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,49.873014,45.406397,43.763603,39.956548,38.929852,40.319777,40.614673,35.022392,40.600974,39.525600,...,52.783821,52.783838,52.783838,52.783821,52.783821,53.515526,54.351566,55.187589,56.023782,56.859992
4439,47.473824,43.007207,41.364476,37.557421,36.530662,37.920587,38.215546,32.623264,38.201784,37.126410,...,55.238849,55.238930,55.238930,55.238849,55.238849,55.532880,55.868754,56.204546,56.540481,56.876496
4440,50.150536,44.510322,43.093867,38.914270,37.649991,38.486820,38.366937,32.373243,38.087553,36.635278,...,55.971056,55.971156,55.971156,55.971056,55.971056,55.877600,55.770701,55.663702,55.556783,55.449965
4441,45.882778,41.925129,40.469642,37.096398,36.186582,37.418126,37.679530,32.724486,37.667281,36.714445,...,56.248125,56.248233,56.248233,56.248125,56.248125,56.291139,56.340164,56.389082,56.438117,56.487259


In [23]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne" in i:
            deme = i.split(".")[1]
            interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({"deme": deme, "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [24]:
test_north = generate_summary_df(ne_diff_summary)

In [25]:
test_north

,deme,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50
0,0,0,41.551583,51.699340,32.173027,44.586715,38.124734
0,0,1,38.039142,48.077472,28.443492,40.881130,34.340671
0,0,2,36.379905,46.338232,26.577614,39.791090,33.202484
0,0,3,35.347937,45.192442,25.367142,38.855383,32.200700
0,0,4,36.979071,46.852645,27.147667,40.106025,33.558772
...,...,...,...,...,...,...,...
0,1,794,57.524441,67.952865,48.715205,59.983030,53.937518
0,1,795,57.413273,65.921932,49.780883,59.543931,54.429862
0,1,796,57.301964,64.508136,50.159818,59.183463,54.544256
0,1,797,57.190777,64.541938,50.453365,58.905889,54.296523


In [26]:
test_north['days'] = test_north.interval.astype(int)
test_north['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - test_north.days.map(timedelta)
test_north.date = test_north.date.astype(str)

In [27]:
line = alt.Chart(test_north).mark_area().encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="transmission rate"), scale = alt.Scale(zero= False)),
    alt.Y2('upper_hpd_log_50'),
    color=alt.Color('deme:N')
).properties(
    width=850,
    height=300
)

band = alt.Chart(test_north).mark_area(
    opacity=0.3
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95'),
    alt.Y2('upper_hpd_log_95'),
    color=alt.Color('deme:N')
).properties(
    width=850,
    height=300
)

band + line

alt.LayerChart(...)

## calculating backward migration rates

In [28]:
def read_in_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
                if "sample" in line.lower():
                    all_cols = line.split("\t")
                    migration_column_indices = []   # list to store column indices
                    rate_scaler_indicies = []
                    error_indicies = []
                    individual_scaler = []
                    mig_rates_key = {}   # dictionary to store the column index to map to column name
                    rate_scaler_key = {}
                    error_key = {}
                    individual_scaler_key = {}
                    counter = 0
                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if col == "immigrationRate.1": #mascot repeats the intro rates which causes an array error. this is done to prevent that
                            counter = counter + 1
                        if ("immigrationRate" or "migrationGLM.1Clock" or "migrationError" or "migrationGLM.1scaler." in col) and (counter <2):
                            migration_column_indices.append(i)

                               
                    # make an empty dictionary to store migration rates and generate dictionary to convert index to name
                    for m in migration_column_indices:
                        name = line.split("\t")[m]
                        mig_rates_key[m] = name
                        mig_rates_dict[name] = []
                    
                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in migration_column_indices:
                        name = mig_rates_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                
                
    return(mig_rates_dict)

In [29]:
migration_rates_f = read_in_migration_rates_mascot(log_file_path)

In [30]:
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)


In [ ]:
mig_df_f

In [ ]:
burnin_percent = 0.1
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
#mig_df_f

4936
4443


In [32]:
def extract_covariate(xml, covar):
    with open(xml, "r") as infile:
            line_number = 0
            for line in infile:
                line_number += 1
                if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                    # use the first line to find the migration rate columns
                    
                    if covar in line:
                        #print(line)
                        s = r"(?<=>)[^<:]+(?=:?<)"
                        match = re.search(s, line)[0]
                        
                        return match
                    
                    
                        


In [33]:
xml = '../../mascot_glm/xmls/glm_mcc_map_randomkc_clusters_combined_new.xml'

In [34]:
npi = extract_covariate(xml, "NPI_dates")
mvmt = extract_covariate(xml, "safegraph_between_total_mvmt")


In [35]:
NPI_dates = npi.split()
NPI_dates = [int(x) for x in NPI_dates]
NPI_dates

safegraph_between_total_mvmt = mvmt.split()
safegraph_between_total_mvmt = [float(x) for x in safegraph_between_total_mvmt]

In [36]:
len(safegraph_between_total_mvmt)

1612

In [37]:
predictors = {"npi_dates": NPI_dates, "mvmt":safegraph_between_total_mvmt }
predict_df = pd.DataFrame(predictors)
predict_df['log_mvmt'] = np.log(predict_df.mvmt)
predict_df['std_log_mvmt'] = (predict_df.log_mvmt - predict_df.log_mvmt.mean())/ predict_df.log_mvmt.std()
#log standarization


In [38]:
predict_df


,npi_dates,mvmt,log_mvmt,std_log_mvmt
0,0,1.397995e+06,14.150550,0.672629
1,0,1.397995e+06,14.150550,0.672629
2,0,1.397995e+06,14.150550,0.672629
3,0,1.397995e+06,14.150550,0.672629
4,0,1.397995e+06,14.150550,0.672629
...,...,...,...,...
1607,0,1.885497e+06,14.449702,2.349775
1608,0,1.885497e+06,14.449702,2.349775
1609,0,1.885497e+06,14.449702,2.349775
1610,0,1.885497e+06,14.449702,2.349775


In [ ]:
mig_rates = {}
counter_n = 0
counter_s = 0
for index_1, row_1 in predict_df.iterrows():
    

    if index_1 %2 == 0:
        mig_rates[str(counter_n) + "_N_to_S_b"] = []
        for index_2, row_2 in mig_df_f.iterrows():
        
            mig_rate_base = (float(row_1.std_log_mvmt)*float(row_2['migrationGLM.1scaler.safegraph_between_total_mvmt']))+ (float(row_1.npi_dates)*float(row_2["migrationGLM.1scaler.NPI_dates"])) + (float(row_2['migrationErrorGLM.1.1'])) 
            mig_rate_f = (math.exp(mig_rate_base)) * (float(row_2['migrationGLM.1Clock']))
            mig_rate_b = mig_rate_f* ((float(row_2["Ne.0."+ str(counter_n)]))/(float(row_2["Ne.1."+ str(counter_n)])))
            mig_rates[str(counter_n) + "_N_to_S_b"].append(mig_rate_b)   
            
        counter_n= counter_n+1
                
    else:
        mig_rates[str(counter_s) + "_S_to_N_b"] = []
        for index_2, row_2 in mig_df_f.iterrows():
        
            mig_rate_base = (float(row_1.std_log_mvmt)*float(row_2['migrationGLM.1scaler.safegraph_between_total_mvmt']))+ (float(row_1.npi_dates)*float(row_2["migrationGLM.1scaler.NPI_dates"])) + (float(row_2['migrationErrorGLM.1.2'])) 
            mig_rate_f = (math.exp(mig_rate_base)) * (float(row_2['migrationGLM.1Clock']))
            mig_rate_b = mig_rate_f* ((float(row_2["Ne.1."+ str(counter_s)]))/(float(row_2["Ne.0."+ str(counter_s)])))
            mig_rates[str(counter_s) + "_S_to_N_b"].append(mig_rate_b)
        counter_s = counter_s +1
    #mig_rates[index_1].append(interval_rate)
    

In [ ]:
mig_df_f

In [ ]:
mr_b_df = pd.DataFrame(mig_rates)

In [ ]:
mr_b_df

In [ ]:
north_mrb =  mr_b_df.filter(regex='S_to_N')
south_mrb =  mr_b_df.filter(regex='N_to_S')

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        #print(i)
        if ("_N_to_S_b" in str(i)) | ("_S_to_N_b" in str(i)):
            #deme = i.split("_")[1]
            interval = i.split("_")[0]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]


            
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [ ]:
north_mrb_df = generate_summary_df(north_mrb)
south_mrb_df = generate_summary_df(south_mrb)

In [ ]:
south_mrb_df['days'] = south_mrb_df.interval.astype(int) 
south_mrb_df['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - south_mrb_df.days.map(timedelta)
south_mrb_df.date = south_mrb_df.date.astype(str)

north_mrb_df['days'] = north_mrb_df.interval.astype(int)
north_mrb_df['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - north_mrb_df.days.map(timedelta)
north_mrb_df.date = north_mrb_df.date.astype(str)

In [ ]:
south_mrb_plot = alt.Chart(south_mrb_df, width = 750).mark_area(interpolate='monotone', opacity = 1.0, color = "orange").encode(
    alt.X('date:T', axis=alt.Axis(title=None, grid=False)),
    alt.Y('upper_hpd_log_95',axis=alt.Axis(title="introductions", grid=False)),
    alt.Y2('lower_hpd_log_95' )
).properties(
    width=800,
    height=300
)

In [ ]:
north_mrb_plot = alt.Chart(north_mrb_df).mark_area(interpolate='monotone', opacity = 1.0).encode(
    alt.X('date:T', axis=alt.Axis(title=None, grid=False)),
    alt.Y('lower_hpd_log_95',axis=alt.Axis(title="introductions", grid=False)),
    alt.Y2('upper_hpd_log_95' )
).properties(
    width=800,
    height=300
)

In [ ]:
north_mrb_plot + south_mrb_plot

### North percent of new cases from intros


In [ ]:
def generate_north_intro_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "S_to_N" in i:
            interval = i.split("_")[0]
            #print(interval)
            #print(i)
            
            immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            try:
                new_df["intro"+".north." + str(interval)] = input_df[i].astype("float") + mig_df["immigrationRate."+str(immigration_interval)].astype('float').map(math.exp)
                #print(input_df[i].astype("float"))
                #print(mig_df["immigrationRate."+str(immigration_interval)].astype('float'))
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [ ]:
intro_north_df = generate_north_intro_df(north_mrb)

In [ ]:
intro_north_df

In [ ]:
def generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "north" in i:
            interval = i.split(".")[2]
            print(interval)
            print(i)
            

            
            try:
                temp_df["total."+ str(interval)] = north_ne_diff["Ne.0.diff." + str(interval)].astype("float") +  input_df[i].astype("float")

                new_df["intro.percent"+".north." + str(interval)] = input_df[i].astype("float").div(temp_df["total."+ str(interval)], axis = 0) 
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [ ]:
percent_df_north = generate_percent_intro_df(intro_north_df)

In [ ]:
percent_df_north

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            #deme = i.split("_")[1]
            interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [ ]:
final_north_df = generate_summary_df(percent_df_north)

In [ ]:
final_north_df

In [ ]:
final_north_df['days'] = final_north_df.interval.astype(int)
final_north_df['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_north_df.days.map(timedelta)
final_north_df = final_north_df[final_north_df.date >"2020-01-31"]
final_north_df.date = final_north_df.date.astype(str)

In [ ]:
final_north_df.to_csv("../data-files/north_percent_intro.csv")

In [ ]:
line1 = alt.Chart(final_north_df).mark_area(interpolate='monotone').encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_50' )
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 < 2)
)

band1 = alt.Chart(final_north_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_95')
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 1.5)
)

band1 + line1

## South

In [ ]:
def generate_south_intro_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "N_to_S" in i:
            interval = i.split("_")[0]
            #print(interval)
            #print(i)
            
            immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            try:
                new_df["intro"+".south." + str(interval)] = input_df[i].astype("float") + mig_df["immigrationRate."+str(immigration_interval)].astype('float').map(math.exp)
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [ ]:
intro_south_df = generate_south_intro_df(south_mrb)

In [ ]:
intro_south_df

In [ ]:
def generate_percent_intro_s_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "south" in i:
            interval = i.split(".")[2]
            
            #print(interval)
            #print(i)
            

            
            try:
                temp_df["total."+ str(interval)] = south_ne_diff["Ne.1.diff." + str(interval)].astype("float") + input_df[i].astype("float")
                #print(south_ne_diff["Ne.1.diff." + str(interval)].astype("float"))
               # print(input_df[i].astype("float"))
                new_df["intro.percent"+".south." + str(interval)] = input_df[i].astype("float").div(temp_df["total."+ str(interval)], axis = 0) 
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [ ]:
percent_df_south = generate_percent_intro_s_df(intro_south_df)

In [ ]:
percent_df_south

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            #deme = i.split("_")[1]
            interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [ ]:
final_south_df = generate_summary_df(percent_df_south)

In [ ]:
final_south_df

In [ ]:
final_south_df['days'] = final_south_df.interval.astype(int)
final_south_df['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_south_df.days.map(timedelta)
final_south_df = final_south_df[final_south_df.date >"2020-01-31"]
final_south_df.date = final_south_df.date.astype(str)

In [ ]:
final_south_df

In [ ]:
final_south_df.to_csv("../data-files/south_percent_intro.csv")

In [ ]:
line2 = alt.Chart(final_south_df).mark_area(interpolate='monotone', opacity = 1 ,color = "#f58518").encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="Percent of cases due to introductions", grid=False)),
    alt.Y2('upper_hpd_log_50' )
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 < 0.6)
)

band2 = alt.Chart(final_south_df).mark_area(
    opacity=0.3, interpolate='monotone', color = "#f58518"
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_95')
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 1)
)

band2 + line2

In [ ]:
band1+ line1+ band2+ line2

In [ ]:
#highlighting important NPIs in WA
data = {'date': [ "2020-03-23", "2020-06-01", "2020-11-18", "2021-02-14"], 'event':[ "Stay at home", "Stay at home lifted", "Closing restaurants", "Reopening restaurants"]}

npidf = pd.DataFrame(data)
npidf.date = pd.to_datetime(npidf.date)

rule = alt.Chart(npidf).mark_rule(
    color="black",
    strokeWidth=2, 
    opacity = 0.3
).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
).properties(
    width=1850,
    height=300
)

text = alt.Chart(npidf).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-135,
    size=10
).encode(
    alt.X('date:T',axis=alt.Axis(title=None)),
    text='event',
    color=alt.value('#000000')
).properties(
    width=1850,
    height=300
)

In [ ]:
band1+ line1+ band2+ line2 + text + rule

In [ ]:
percent_case_intro = band1+ line1+ band2+ line2 + text + rule

In [ ]:
percent_case_intro.save("../figures/percent_case_intro_random.html")

## repeating percent intro but without introductions from outside

In [ ]:
def north_generate_percent_intro_df(input_df):
    
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "S_to_N" in i:
            interval = i.split("_")[0]


            
            try:
                temp_df["total."+ str(interval)] = north_ne_diff["Ne.0.diff." + str(interval)].astype("float") +  input_df[i].astype("float")

                new_df["intro.percent"+".north." + str(interval)] = input_df[i].astype("float").div(temp_df["total."+ str(interval)], axis = 0) 
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [ ]:
def south_generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "N_to_S" in i:
            interval = i.split("_")[0]
            
            
            try:
                temp_df["total."+ str(interval)] = south_ne_diff["Ne.1.diff." + str(interval)].astype("float") + input_df[i].astype("float")
                #print(south_ne_diff["Ne.1.diff." + str(interval)].astype("float"))
               # print(input_df[i].astype("float"))
                new_df["intro.percent"+".south." + str(interval)] = input_df[i].astype("float").div(temp_df["total."+ str(interval)], axis = 0) 
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            #deme = i.split("_")[1]
            interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [ ]:
percent_df_north_within = north_generate_percent_intro_df(north_mrb)
percent_df_south_within = south_generate_percent_intro_df(south_mrb)

In [ ]:
final_south_df_within = generate_summary_df(percent_df_south_within)
final_north_df_within = generate_summary_df(percent_df_north_within)

In [ ]:
final_south_df_within['days'] = final_south_df_within.interval.astype(int)
final_south_df_within['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_south_df_within.days.map(timedelta)
final_south_df_within = final_south_df_within[final_south_df_within.date >"2020-01-31"]
final_south_df_within.date = final_south_df_within.date.astype(str)

final_north_df_within['days'] = final_north_df_within.interval.astype(int)
final_north_df_within['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_north_df_within.days.map(timedelta)
final_north_df_within = final_north_df_within[final_north_df_within.date >"2020-01-31"]
final_north_df_within.date = final_north_df_within.date.astype(str)


In [ ]:
final_south_df_within['region'] = "South King County"
final_north_df_within['region'] = "North King County"

In [ ]:
line2 = alt.Chart(final_south_df_within).mark_area(interpolate='monotone', opacity = 1 ,color = "#f58518").encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="Percent of cases due to introductions", grid=False)),
    alt.Y2('upper_hpd_log_50' )
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 < 0.6)
)

band2 = alt.Chart(final_south_df_within).mark_area(
    opacity=0.3, interpolate='monotone', color = "#f58518"
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_95')
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 1)
)

band2 + line2

In [ ]:
line3 = alt.Chart(final_north_df_within).mark_area(interpolate='monotone', opacity = 1 ).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="Percent of cases due to introductions", grid=False)),
    alt.Y2('upper_hpd_log_50' )
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 < 1)
)

band3 = alt.Chart(final_north_df_within).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_95')
).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 1.5)
)

band3 + line3

In [ ]:
band2 + line2 + band3 + line3

## working on doing Rt calculations based on percent intro

In [ ]:
def generate_south_rt_without_intro_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[3]
            
            #print(interval)
            #print(i)
            
            #immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            #try:
            new_df["rt"+".south." + str(interval)] = (input_df[i].astype("float") * (1- percent_df_south["intro.percent.south."+str(interval)].astype("float")))/ uninfectious_rate
            
            
            #except KeyError:
             #   pass 
            
            
    return(new_df)

In [ ]:
def generate_south_rt_within_only_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[3]
            
            #print(interval)
            #print(i)
            
            #immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            #try:
            new_df["rt"+".south." + str(interval)] = (input_df[i].astype("float") * (1- percent_df_south_within["intro.percent.south."+str(interval)].astype("float")))/ uninfectious_rate
            
            
            #except KeyError:
             #   pass 
            
            
    return(new_df)

In [ ]:
def generate_south_rt_with_intro_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[3]
            
            #print(interval)
            #print(i)
            
            #immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            #try:
            new_df["rt"+".south." + str(interval)] = (input_df[i].astype("float"))/ uninfectious_rate
            
            
            #except KeyError:
             #   pass 
            
            
    return(new_df)

In [ ]:
south_rt_no_intro_df = generate_south_rt_without_intro_df(south_ne_diff)
south_rt_with_intro_df = generate_south_rt_with_intro_df(south_ne_diff)
south_rt_within_only_df = generate_south_rt_within_only_df(south_ne_diff)

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "rt" in i:
            #deme = i.split("_")[1]
            interval = i.split(".")[2]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [ ]:
final_south_no_intro_rt = generate_rt_summary_df(south_rt_no_intro_df)
final_south_with_intro_rt = generate_rt_summary_df(south_rt_with_intro_df)
final_south_within_only_rt = generate_rt_summary_df(south_rt_within_only_df)

In [ ]:
final_south_no_intro_rt

In [ ]:
def generate_north_rt_no_intro_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[3]
            
            #print(interval)
            #print(i)
            
            #immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            #try:
            new_df["rt"+".north." + str(interval)] = (input_df[i].astype("float") * (1- percent_df_north["intro.percent.north."+str(interval)].astype("float")))/ uninfectious_rate
            
            
            #except KeyError:
             #   pass 
            
            
    return(new_df)

In [ ]:
def generate_north_rt_within_only_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[3]
            
            #print(interval)
            #print(i)
            
            #immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            #try:
            new_df["rt"+".north." + str(interval)] = (input_df[i].astype("float") * (1- percent_df_north_within["intro.percent.north."+str(interval)].astype("float")))/ uninfectious_rate
            
            
            #except KeyError:
             #   pass 
            
            
    return(new_df)

In [ ]:
def generate_north_rt_with_intro_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[3]
            
            #print(interval)
            #print(i)
            
            #immigration_interval = math.ceil((int(interval)+1)/14)
            #print(immigration_interval)

            
            #try:
            new_df["rt"+".north." + str(interval)] = (input_df[i].astype("float") )/ uninfectious_rate
            
            
            #except KeyError:
             #   pass 
            
            
    return(new_df)

In [ ]:
north_rt_no_intro_df = generate_north_rt_no_intro_df(north_ne_diff)
north_rt_with_intro_df = generate_north_rt_with_intro_df(north_ne_diff)
north_rt_within_only_df = generate_north_rt_within_only_df(north_ne_diff)

In [ ]:
final_north_no_intro_rt = generate_rt_summary_df(north_rt_no_intro_df)
final_north_with_intro_rt = generate_rt_summary_df(north_rt_with_intro_df)
final_north_within_only_rt = generate_rt_summary_df(north_rt_within_only_df)

In [ ]:
final_north_no_intro_rt

In [ ]:
final_south_no_intro_rt['days'] = final_south_no_intro_rt.interval.astype(int) 
final_south_no_intro_rt['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_south_no_intro_rt.days.map(timedelta)
final_south_no_intro_rt.date = final_south_no_intro_rt.date.astype(str)

final_south_with_intro_rt['days'] = final_south_with_intro_rt.interval.astype(int) 
final_south_with_intro_rt['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_south_with_intro_rt.days.map(timedelta)
final_south_with_intro_rt.date = final_south_with_intro_rt.date.astype(str)

final_south_within_only_rt['days'] = final_south_within_only_rt.interval.astype(int) 
final_south_within_only_rt['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_south_within_only_rt.days.map(timedelta)
final_south_within_only_rt.date = final_south_within_only_rt.date.astype(str)

final_north_within_only_rt['days'] = final_north_within_only_rt.interval.astype(int) 
final_north_within_only_rt['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_north_within_only_rt.days.map(timedelta)
final_north_within_only_rt.date = final_north_within_only_rt.date.astype(str)

final_north_no_intro_rt['days'] = final_north_no_intro_rt.interval.astype(int) 
final_north_no_intro_rt['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_north_no_intro_rt.days.map(timedelta)
final_north_no_intro_rt.date = final_north_no_intro_rt.date.astype(str)

final_north_with_intro_rt['days'] = final_north_with_intro_rt.interval.astype(int) 
final_north_with_intro_rt['date'] = dt.strptime("2022-03-06",  "%Y-%m-%d") - final_north_with_intro_rt.days.map(timedelta)
final_north_with_intro_rt.date = final_north_with_intro_rt.date.astype(str)



In [ ]:
final_south_no_intro_rt['Contribution'] = "Local"
final_south_with_intro_rt['Contribution'] = "Outside King County"
final_south_within_only_rt['Contribution'] = "Other King County Region"


final_north_no_intro_rt['Contribution'] = "Local"
final_north_with_intro_rt['Contribution'] = "Outside King County"
final_north_within_only_rt['Contribution'] = "Other King County Region"


In [ ]:
combined_rt_south = pd.concat([ final_south_with_intro_rt, final_south_within_only_rt, final_south_no_intro_rt], ignore_index=True)
combined_rt_north = pd.concat([ final_north_with_intro_rt, final_north_within_only_rt, final_north_no_intro_rt], ignore_index=True)

In [ ]:
combined_rt_north.to_csv("../data-files/combined_rt_north.csv")
combined_rt_south.to_csv("../data-files/combined_rt_south.csv")

In [ ]:
combined_rt_south

In [ ]:
stream_south = alt.Chart(combined_rt_south, title = "South King County").mark_area(interpolate='monotone', opacity = .7 ,color = "#f58518", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('mean_percent',axis=alt.Axis(title="", grid=False), stack = False, scale=alt.Scale(domain=(0, 2.5))),
    #alt.Y2('upper_hpd_log_50' ), 
    alt.Color('Contribution:N', scale=alt.Scale(domain = ['Local', "Other King County Region", "Outside King County"], range = [ "#4c90c0", "#ceb541", "#df4327"]),
             legend=alt.Legend(offset = -160, labelFontSize = 12, titleFontSize = 12))
).properties(
    width=400,
    height=300
).transform_filter(
    (datum.mean_percent >0) & (datum.mean_percent < 2.5)
)

In [ ]:
stream_north = alt.Chart(combined_rt_north, title = "North King County").mark_area(interpolate='monotone', opacity = .7 ,color = "#f58518", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('mean_percent',axis=alt.Axis(title="Local Rt", grid=False),stack = False, scale=alt.Scale(domain=(0, 2.5))),
    #alt.Y2('upper_hpd_log_50' ), 
    alt.Color('Contribution:N', scale=alt.Scale(domain = ['Local', "Other King County Region", "Outside King County"], range = ["#4c90c0", "#ceb541", "#df4327"]))
).properties(
    width=400,
    height=300
).transform_filter(
    (datum.mean_percent >0) & (datum.mean_percent < 2.5)
)

In [ ]:
one_line = alt.Chart(pd.DataFrame({'y': [1.0]})).mark_rule(strokeDash=[1,1]).encode(y='y')

In [ ]:
stream_south

In [ ]:
rt_streamplot = ((stream_north +one_line) | (stream_south+one_line)) 
rt_streamplot.save("../figures/rt_streamplot_random.html")
rt_streamplot

In [ ]:
line2 = alt.Chart(final_south_no_intro_rt).mark_area(interpolate='monotone', opacity = 1 ,color = "#f58518").encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="Rt accounting for % intro", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_50' )
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 < 3)
)
band2 = alt.Chart(final_south_no_intro_rt).mark_area(
    opacity=0.3, interpolate='monotone', color = "#f58518"
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95')
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 3)
)

band2 + line2

In [ ]:
line1 = alt.Chart(final_north_no_intro_rt).mark_area(interpolate='monotone').encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="", grid=False)),#, scale=alt.Scale(domain=(0.4, 1.3)) ),
    alt.Y2('upper_hpd_log_50' )
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 <3)
)

band1 = alt.Chart(final_north_no_intro_rt).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95')
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 3)
)

band1 + line1

In [ ]:
band2 + line2 + band1 + line1  + north_case_rt +south_case_rt

In [ ]:
final_north_no_intro_rt['Location'] = "North King County"
final_south_no_intro_rt['Location'] = "South King County"

combined_no_intro_rt = pd.concat([ final_north_no_intro_rt, final_south_no_intro_rt], ignore_index=True)


In [ ]:
line4 = alt.Chart(combined_no_intro_rt).mark_area(interpolate='monotone').encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50',axis=alt.Axis(title="", grid=False)),#, scale=alt.Scale(domain=(0.4, 1.3)) ),
    alt.Y2('upper_hpd_log_50' ), 
    alt.Color('Location:N',legend=alt.Legend(offset = -160, labelFontSize = 12, titleFontSize = 12))
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50 >0) & (datum.upper_hpd_log_50 <3)
)

band4 = alt.Chart(combined_no_intro_rt).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="Date", grid=False)),
    alt.Y('lower_hpd_log_95', axis=alt.Axis(title="", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95'),
    alt.Color('Location:N')
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95 >0) & (datum.upper_hpd_log_95 < 3)
)

comb_rt_no_intro = band4 + line4
comb_rt_no_intro

## here we repeat the above calculations but with case data rather than Ne

In [ ]:
cases = pd.read_csv('~/Desktop/gitrepos/ncov-king-county/analysis/data-files/absolute_cases_by_puma.csv', parse_dates = [2])


In [ ]:
cases

In [ ]:
cases.Positives[cases.Positives == 0] = 0.1

In [ ]:
n_cases = cases[cases.region == "nKC"]
s_cases = cases[cases.region == "sKC"]

n_cases_no_intro = cases[cases.region == "nKC"]
s_cases_no_intro = cases[cases.region == "sKC"]


In [ ]:
final_north_df.date = pd.to_datetime(final_north_df.date)
final_south_df.date = pd.to_datetime(final_south_df.date)

In [ ]:
n_cases['diff'] = np.nan
n_cases.Positives = n_cases.Positives.astype(float)
for index, row in n_cases.iterrows():
    forward_date = row.Date + pd.Timedelta(7, unit='d')

    try:
        n_cases.loc[index, "diff"] = ((((365/7)* (np.log(n_cases['Positives'][n_cases['Date'] == forward_date].item()) - np.log(n_cases.Positives[n_cases.Date == row.Date].item()))) + 52.1429)*(1-final_north_df.mean_percent[final_north_df.date == row.Date].item()))/52.1429
    except ValueError:
        continue


In [ ]:
s_cases['diff'] = np.nan
s_cases.Positives = s_cases.Positives.astype(float)
for index, row in s_cases.iterrows():
    forward_date = row.Date + pd.Timedelta(7, unit='d')
    try:
        s_cases.loc[index, "diff"] = ((((365/7)* (np.log(s_cases['Positives'][s_cases['Date'] == forward_date].item()) - np.log(s_cases.Positives[s_cases.Date == row.Date].item()))) + 52.1429)*(1-final_south_df.mean_percent[final_south_df.date == row.Date].item()))/52.1429
    except ValueError:
        continue

In [ ]:
south_ne_diff

In [ ]:
n_cases

In [ ]:
south_case_rt = alt.Chart(s_cases).mark_line(interpolate='monotone', color = "black", strokeDash=[2,2]).encode(
    alt.X('Date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('diff:Q',axis=alt.Axis(title="", grid=False))).properties(
    width=850,
    height=300).transform_filter((datum.diff <5) & (datum.diff >0))
south_case_rt

In [ ]:
north_case_rt = alt.Chart(n_cases).mark_line(interpolate='monotone', color ="black").encode(
    alt.X('Date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('diff:Q',axis=alt.Axis(title="Rt accounting for % intro", grid=False))).properties(
    width=850,
    height=300).transform_filter((datum.diff <5) & (datum.diff >0))
north_case_rt

In [ ]:
north_case_rt +south_case_rt

In [ ]:
n_cases['Location'] = "North King County"
s_cases['Location'] = "South King County"

combined_case_rt = pd.concat([n_cases, s_cases], ignore_index=True)


In [ ]:
comb_case_rt = alt.Chart(combined_case_rt).mark_line(interpolate='monotone', color ="black").encode(
    alt.X('Date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('diff:Q',axis=alt.Axis(title="Rt accounting for % intro", grid=False)),
    alt.StrokeDash("Location:N",legend=alt.Legend(offset = -160, labelFontSize = 12, titleFontSize = 12))).properties(
    width=850,
    height=300).transform_filter((datum.diff <5) & (datum.diff >0))
comb_case_rt

In [ ]:
n_cases_no_intro['diff'] = np.nan
n_cases_no_intro.Positives = n_cases_no_intro.Positives.astype(float)
for index, row in n_cases_no_intro.iterrows():
    forward_date = row.Date + pd.Timedelta(7, unit='d')

    try:
        n_cases_no_intro.loc[index, "diff"] = ((((365/7)* (np.log(n_cases_no_intro['Positives'][n_cases_no_intro['Date'] == forward_date].item()) - np.log(n_cases_no_intro.Positives[n_cases_no_intro.Date == row.Date].item()))) + 52.1429))/52.1429
    except ValueError:
        continue


In [ ]:
s_cases_no_intro['diff'] = np.nan
s_cases_no_intro.Positives = s_cases_no_intro.Positives.astype(float)
for index, row in s_cases_no_intro.iterrows():
    forward_date = row.Date + pd.Timedelta(7, unit='d')
    try:
        s_cases_no_intro.loc[index, "diff"] = ((((365/7)* (np.log(s_cases_no_intro['Positives'][s_cases_no_intro['Date'] == forward_date].item()) - np.log(s_cases_no_intro.Positives[s_cases_no_intro.Date == row.Date].item()))) + 52.1429))/52.1429
    except ValueError:
        continue

In [ ]:
south_case_rt_no_intro = alt.Chart(s_cases_no_intro).mark_line(interpolate='monotone', color = "black", strokeDash=[2,2]).encode(
    alt.X('Date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('diff:Q',axis=alt.Axis(title="Rt without accounting for % intro", grid=False))).properties(
    width=850,
    height=300).transform_filter((datum.diff <10) & (datum.diff >0))
south_case_rt_no_intro

In [ ]:
north_case_rt_no_intro = alt.Chart(n_cases_no_intro).mark_line(interpolate='monotone', color = "black").encode(
    alt.X('Date:T', axis=alt.Axis(title="Date", grid=False,format="%B %Y")),
    alt.Y('diff:Q',axis=alt.Axis(title="", grid=False))).properties(
    width=850,
    height=300).transform_filter(datum.diff <5)
north_case_rt_no_intro

In [ ]:
north_case_rt_no_intro +south_case_rt_no_intro

In [ ]:
local_rt_combined = comb_rt_no_intro + comb_case_rt
local_rt_combined

In [ ]:
local_rt_combined.save("../figures/local_rt_combined.html")